Instructor: Piyawat Chuangkrud

# Install Requirements

In [ ]:
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl==0.7.10
!pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U datasets==2.17.0
!pip3 install -q -U transformers==4.38.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This beha

# Import Library

In [ ]:
import torch

In [ ]:
torch.cuda.current_device()

0

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

import torch
import torch.nn as nn

import transformers
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          TrainingArguments,
                          pipeline,
                          logging)
from datasets import Dataset
from peft import LoraConfig, PeftConfig
import bitsandbytes as bnb
from trl import SFTTrainer

from sklearn.metrics import (accuracy_score,
                             classification_report,
                             confusion_matrix)
from sklearn.model_selection import train_test_split

In [ ]:
from datasets import load_dataset

# Load Dataset

In [ ]:
data = load_dataset('sepidmnorozy/Thai_sentiment')

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 8103
    })
    validation: Dataset({
        features: ['label', 'text'],
        num_rows: 1153
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 2344
    })
})

# Data Proprocess

In [ ]:
def generate_prompt(data_point):
    return f"""
            Analyze the sentiment of the tweet enclosed in square brackets,
            determine if it is positive or negative, and return the answer as
            the corresponding sentiment label "positive" or  "negative"

            [{data_point["text"]}] = {data_point["label"]}
            """.strip()

def generate_test_prompt(data_point):
    return f"""
            Analyze the sentiment of the tweet enclosed in square brackets,
            determine if it is positive or negative, and return the answer as
            the corresponding sentiment label "positive" or  "negative"

            [{data_point["text"]}] =

            """.strip()

## Convert to Pandas

In [ ]:
test = data['test'].to_pandas()
validation = data['validation'].to_pandas()
train = data['train'].to_pandas()

In [ ]:
def convert(x):
    if x==1:
        return 'positive'
    elif x==0:
        return 'negative'
    else:
        return 'none'

In [ ]:
train['label'] = train['label'].apply(lambda x: convert(x))

In [ ]:
validation['label'] = validation['label'].apply(lambda x: convert(x))

In [ ]:
test['label'] = test['label'].apply(lambda x: convert(x))

## Apply Prompt

In [ ]:
X_train = pd.DataFrame(train.apply(generate_prompt, axis=1),
                       columns=["text"])
X_eval = pd.DataFrame(validation.apply(generate_prompt, axis=1),
                      columns=["text"])

In [ ]:
y_true = test.label
X_test = pd.DataFrame(test.apply(generate_test_prompt, axis=1), columns=["text"])

In [ ]:
train_data = Dataset.from_pandas(X_train)
eval_data = Dataset.from_pandas(X_eval)

In [ ]:
print(train_data['text'][0])

Analyze the sentiment of the tweet enclosed in square brackets,
            determine if it is positive or negative, and return the answer as
            the corresponding sentiment label "positive" or  "negative"

            [อยากให้มี ขนมปังชีส ไอศครีมนมสด หอมทอด แซนวิชอร่อยๆ] = positive


In [ ]:
print(eval_data['text'][0])

Analyze the sentiment of the tweet enclosed in square brackets,
            determine if it is positive or negative, and return the answer as
            the corresponding sentiment label "positive" or  "negative"

            [เกลียดการขายเบียร์ช้าง] = negative


# Evaluation Function

In [ ]:
def evaluate(y_true, y_pred):

    labels = ['positive',  'negative']
    mapping = {'positive': 1, 'negative': 0, 'none':1,}
    def map_func(x):
        return mapping.get(x, 1)

    y_true = np.vectorize(map_func)(y_true)
    y_pred = np.vectorize(map_func)(y_pred)

    # Calculate accuracy
    accuracy = accuracy_score(y_true=y_true, y_pred=y_pred)
    print(f'Accuracy: {accuracy:.3f}')

    # Generate accuracy report
    unique_labels = set(y_true)  # Get unique labels

    for label in unique_labels:
        label_indices = [i for i in range(len(y_true))
                         if y_true[i] == label]
        label_y_true = [y_true[i] for i in label_indices]
        label_y_pred = [y_pred[i] for i in label_indices]
        accuracy = accuracy_score(label_y_true, label_y_pred)
        print(f'Accuracy for label {label}: {accuracy:.3f}')

    # Generate classification report
    class_report = classification_report(y_true=y_true, y_pred=y_pred)
    print('\nClassification Report:')
    print(class_report)

    # Generate confusion matrix
    conf_matrix = confusion_matrix(y_true=y_true, y_pred=y_pred, labels=[0, 1])
    print('\nConfusion Matrix:')
    print(conf_matrix)

# Load Model

#### huggingface-cli login via token

In [ ]:
!huggingface-cli login --token YOUR_TOKEN_ID

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


## Load Model from Huggingface

In [ ]:
model_name = "google/gemma-2b"

compute_dtype = getattr(torch, "float16")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,
)

model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
model

# Predict Function

In [ ]:
def predict(X_test, model, tokenizer):
    y_pred = []
    for i in tqdm(range(len(X_test))):
        prompt = X_test.iloc[i]["text"]
        input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
        outputs = model.generate(**input_ids, max_new_tokens=1, temperature=0.0)
        result = tokenizer.decode(outputs[0])
        answer = result.split("=")[-1].lower()
        if "positive" in answer:
            y_pred.append("positive")
        elif "negative" in answer:
            y_pred.append("negative")
        elif "neutral" in answer:
            y_pred.append("neutral")
        else:
            y_pred.append("none")
    return y_pred

# Zero-shot Classification

In [ ]:
y_pred = predict(X_test , model, tokenizer)

# Evaluate Zero-Shot Model

In [ ]:
evaluate(y_true, y_pred)


# Finetune Model

In [ ]:
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules="all-linear",
)

training_arguments = TrainingArguments(
    output_dir="logs",
    num_train_epochs=2,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    gradient_checkpointing=False,
    optim="paged_adamw_32bit",
    save_steps=0,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="cosine",
    # report_to="tensorboard",
    do_eval=False,
    evaluation_strategy="no",
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    peft_config=peft_config,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
    max_seq_length=512,

)

In [ ]:
trainer.train()

## Evaluate Finetune Model

In [ ]:
y_pred = predict(X_test, model, tokenizer)
evaluate(y_true, y_pred)

# Push Adapter to Hub

In [ ]:
trainer.model.save_pretrained("adapter-name")

In [ ]:
trainer.model.push_to_hub("your-hub-repo")

# Inference

In [ ]:
def inference(text, model, tokenizer):
    prompt = f"""
            Analyze the sentiment of the tweet enclosed in square brackets,
            determine if it is positive or negative, and return the answer as
            the corresponding sentiment label "positive" or  "negative"

            [{text}] =

            """.strip()
    input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**input_ids, max_new_tokens=1, temperature=0.0)
    result = tokenizer.decode(outputs[0])
    answer = result.split("=")[-1].lower().strip()

    return answer

In [ ]:
inference('วันนี้อากาศดีจัง', model, tokenizer)